前面使用了k均值聚类来进行测试,虽然说效果貌似还行,但是总觉得不是这样的,我查看过分类出来的group的成员的数据,发现它们其实相关性不大,其实仔细想一想,也对,前面的算法主要是使用pearson距离或者欧几里德距离来度量两个向量的相似度,这样的话,分类出来的数据自然是彼此之间的pearson距离或者欧几里德距离最小的,为了使得我们分类出来的数据满足我们的要求,我们要改动一下相似度计算函数:

两组向量相似的话,那么它们每天应该有差不多的运动时间,差不多的静止时间,甚至于每天运动的时段都有所相似,根据这些点,我们可以调整一些参数,控制相似度:

In [ ]:
import pymssql
import random
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

def conn_db(server="127.0.0.1", user="sa", pwd="123", db="moveState"):
    """用于连接数据库"""
    return pymssql.connect(server, user, pwd, db)

def similarity(v1, v2):
    '''
    用于计算两个向量的相似度,这个相似度大致是这样来计算的,3月一共有31天,1天有24个小时,
    如果在同一天里每个小时的运动状态一致的数目越大,那么相似度越高,一天的权值是1/31,可以通过这种方法
    来衡量两辆车运动状态的相似度
    '''
    def simi_day(d1, d2):
        counter = 0
        for i in range(24):
            if d1[i] == d2[i]:
                counter += 1
        return counter / 24.0

    def simi_day_ex(d1, d2):
        '''
        用另外一种评价标准,因为不运动的时间占据了车辆的绝大多数时间,所以即使相似度不大的两种车
        因为这个缘故用上面的评价标准的话,也会变得很相似,所以要改进相似度的评价
       '''
        simi_t = 0.0 # 一天中同处在运动状态的小时的计数
        diff_t = 0.0 # 一天内两车处于不相同的运动状态的小时的次数
        # 都处在静止状态就排除掉了
        for i in range(24):
            if d1[i] == d2[i] == 1:
                simi_t += 1
            elif d1[i] != d2[i]:
                diff_t += 1
        divisor = simi_t + diff_t
        if  divisor == 0:
            return 0.25
        else:
            return (simi_t / divisor) * 0.75 + ((24 - divisor) / 24) * 0.25

    simi = 0.0
    for day in range(31):
        start = day * 24
        end = start + 24
        simi += simi_day_ex(v1[start:end], v2[start:end])
    return simi

def kcluster(rows, distance=similarity, k=10):
    """k均值聚类"""
    # 随机创建k个中心点
    # random.random()返回一个在[0.0, 1.0)之间的伪随机数
    def rand():
        if random.random() >= 0.5:
            return 1.0
        else:
            return 0.0
    n = len(rows[0]) # 向量的长度
    clusters = [[rand() for i in range(n)] for j in range(k)]

    lastmatches = None
    for t in range(25): # 最多迭代100次
        print('Iteration %d' % t)
        bestmatches = [[] for i in range(k)]
        # 在每一行中寻找距离最近的中心点
        for j in range(len(rows)):
            row = rows[j]
            bestmatch = 0
            for i in range(k):
                d = distance(clusters[i], row)
                if d < distance(clusters[bestmatch], row): bestmatch = i
            bestmatches[bestmatch].append(j) # 将行的行号加入
        # 如果结果与上一次相同,则整个过程结束
        if bestmatches == lastmatches: break
        lastmatches = bestmatches

        # 将中心点移到其所有成员的平均位置
        for i in range(k):
            avgs = [0.0] * n
            if len(bestmatches[i]) > 0:
                for rowid in bestmatches[i]:
                    for m in range(n): #
                        avgs[m] += rows[rowid][m] # 将rows[rowid]对应位置的

                for j in range(n): # 其实就是求一下平均值
                    avgs[j] /= len(bestmatches[i])
                    if avgs[j] >= 0.35:
                        avgs[j] = 1
                    else:
                        avgs[j] = 0
                clusters[i] = avgs
    return bestmatches

# 接下来要使用迭代器获得数据
def get_next_vec(conn):
    """获得数据向量"""
    def clean_data(item):
        if item == None:
            return 0
        else:
            return item
    cursor = conn.cursor()
    cursor.execute("select * from MoveState03")
    row = cursor.fetchone()
    while row:
        # 在这里,我们要对数据进行一定程度的清理,具体是将NULL变为0
        yield (row[0], list(map(clean_data, row[1:])))
        row = cursor.fetchone()

if __name__ == '__main__':
    conn = conn_db()
    count = 0
    rows = {}
    # 获取20辆车的数据
    size = 380
    for item in get_next_vec(conn):
        id, vec = item
        rows[id] = vec
        #print(vec)
        count = count + 1
        if count >= size: break
    r = [v for x, v in rows.items()]
    print("100 car's data got!")
    bestmatch = kcluster(r)
    i = 1
    with open("res.txt", mode='w') as f:
        for group in bestmatch:
            print("group %02d: %s" %(i, group))
            f.write("group %02d:\n" % i)
            for index in group:
                f.write("%04d -> %s\n" %(index, r[index]))
            i = i + 1



相关系数的调整花费了我挺长的一段时间,刚开始的时候,一大堆的数据总是收敛到一个类里面,但是经过微微调整之后,效果有所提升,然后查看生成的txt文件,发现分类基本满足了要求,从稀疏度,具体的活动时间等指标可以看出来.